In [ ]:
#Added mass+Validation

'''Added Mass theory section was written with reference to Farnaz Rezvany's work, 
available at: https://github.com/frh-uio/MEK4420/tree/master/MandatoryAssignment1 and 
(Newman, J.N. 2017, Marine Hydrodynamics, The Massachusetts Institute of Technology, United States of America)'''

'''reference as MEK4420'''

def added_mass(panels):
    xc, yc, S, sx, sy = panels.get_array('xc','yc','S','sx','sy') # Retrieve gamma, xc, S, sx, sy
    N = len(panels.panels)  
    A = np.zeros((N, N))    # Matrix for storing values achieved from the lhs of the equation
    B11 = np.zeros(N)       # Array(vector) for storing values for the rhs of the equation
    B22 = np.zeros(N)
    B66 = np.zeros(N)

    dS = 2*S                # dS is the length of the panel, and S is half panel
    n1 = -sy
    n2 = sx
    n6 = xc * n2 -yc * n1

    x = np.empty(N+1)
    x[0] = xc[0] - S[0]*sx[0]
    x[1:] = xc + S*sx

    y = np.empty(N+1)
    y[0] = yc[0] - S[0]*sx[0]
    y[1:] = yc + S*sy

    for i in range(N):      #Array transpose to get [x,y] position of a point on the circumference
        r1 = np.linalg.norm(np.array([x[:-1], y[:-1]]).T - np.array([xc[i], yc[i]]), axis=1)    
        r2 = np.linalg.norm(np.array([x[1:], y[1:]]).T - np.array([xc[i], yc[i]]), axis=1)

        '''Opening angle of segment S(i)'''
        theta = -np.arccos( (dS**2 - r2**2 - r1**2)/(-2*r2*r1) )
        theta[np.isnan(theta)] = 0 

        '''Calculation of h coeffecient'''
        h11 = (np.log(r1) + np.log(r2))*0.5*dS
        h22 = (np.log(r1) + np.log(r2))*0.5*dS
        h66 = (np.log(r1) + np.log(r2))*0.5*dS

        '''Adds the angles to the matrix A'''
        A[i] = theta                    # N matrices that are NxN
        np.fill_diagonal(A, -np.pi)     #replace diagonal entries with -pi

        '''Adds rhs to the B-arrays'''
        B11[i] = sum(n1*h11)    
        B22[i] = sum(n2*h22)
        B66[i] = sum(n6*h66)
    '''Calculates phi for the three directions'''
    phi11 = np.linalg.solve(A, B11)   # Solve the linear matrix equation A*phi=B
    phi22 = np.linalg.solve(A, B22)
    phi66 = np.linalg.solve(A, B66)

    '''Calculations of added mass coefficient'''
    m11 = sum(phi11*n1*dS) 
    m12 = sum(phi11*n2*dS)
    m21 = sum(phi22*n1*dS)
    m22 = sum(phi22*n2*dS)
    m66 = sum(phi66*n6*dS)

    if m12-m21>0.01:
      print('warning:added mass is not semmytric ')
    else:
      return m11, m22


In [ ]:
#jack's added mass checking for the hulls
# create geometry for section through hulls

def panels_trans(breadth=1, gap=1, N = 64):
    spacing = gap+breadth
    t_c=0.125
    x1, y1 = transverse_section(breadth, 64, ycen=spacing/2)
    x2, y2 = transverse_section(breadth, 64, ycen=-spacing/2)
    top    = panelize(x1, y1)
    bot    = panelize(x2, y2)
    both   = concatenate(top, bot)
    return(top, bot, both)

def AUV_added_mass():
    top, bot, both = panels_trans(breadth=1, gap=1, N=64)
    both.solve_gamma_O2(np.pi/4)
    m11, m22= added_mass(both)
    return m11, m22

m11_AUV,m22_AUV=AUV_added_mass()
print('added mass coefficient for AUV are: m11: {}, m22: {}'.format(m11_AUV,m22_AUV))